In [1]:
from collections import defaultdict, Counter
import re
from typing import List, Tuple, Dict

In [2]:
class SyllableBPE:
    def __init__(self, num_merges: int = 1000):
        self.num_merges = num_merges
        self.merge_rules: List[Tuple[str, str]] = []
        self.vocab: Dict[Tuple[str, ...], int] = {}

    def get_vocab(self, corpus: List[str], syllable_segmenter) -> Dict[Tuple[str, ...], int]:
        vocab = Counter()
        for line in corpus:
            syllables = syllable_segmenter(line)
            word = tuple(syllables + ['</w>'])  # Use end-of-word marker
            vocab[word] += 1
        return vocab

    def get_pair_freqs(self, vocab: Dict[Tuple[str, ...], int]) -> Dict[Tuple[str, str], int]:
        pairs = defaultdict(int)
        for word, freq in vocab.items():
            for i in range(len(word) - 1):
                pairs[(word[i], word[i + 1])] += freq
        return pairs

    def merge_vocab(self, pair_to_merge: Tuple[str, str], vocab: Dict[Tuple[str, ...], int]) -> Dict[Tuple[str, ...], int]:
        pattern = re.compile(r'(?<!\S)' + re.escape(pair_to_merge[0]) + r'\s+' + re.escape(pair_to_merge[1]) + r'(?!\S)')
        new_vocab = {}
        for word, freq in vocab.items():
            word_str = ' '.join(word)
            new_word_str = pattern.sub(pair_to_merge[0] + pair_to_merge[1], word_str)
            new_word = tuple(new_word_str.split())
            new_vocab[new_word] = freq
        return new_vocab

    def train(self, corpus: List[str], syllable_segmenter):
        self.vocab = self.get_vocab(corpus, syllable_segmenter)
        for _ in range(self.num_merges):
            pairs = self.get_pair_freqs(self.vocab)
            if not pairs:
                break
            best_pair = max(pairs, key=pairs.get)
            self.merge_rules.append(best_pair)
            self.vocab = self.merge_vocab(best_pair, self.vocab)

    def encode(self, text: str, syllable_segmenter) -> List[str]:
        syllables = syllable_segmenter(text) + ['</w>']
        tokens = list(syllables)
        for a, b in self.merge_rules:
            i = 0
            while i < len(tokens) - 1:
                if tokens[i] == a and tokens[i + 1] == b:
                    tokens[i:i+2] = [a + b]
                else:
                    i += 1
        return tokens

In [7]:
# Sample Burmese corpus (as space-separated sentences)
corpus = [
    "ပ ညာ ရေး ဝန် ကြီး ဌာ န သည် အ ထက် တန်း ကျောင်း များ တွင် သင် ကြား မှု အ တွက် သင် ကြား ရေး အ ထောက် အ ကူ ပြု စာ အုပ် များ ကို ထုတ် ဝေ ခဲ့ သည်", 
    "တော် လှန် ရေး အ ထောက် အ ကူ အ တွက် သင် ကြား သည်",
    "ဗိုလ် ချုပ် က တော် ကောက် သည်",
]

# Dummy syllable segmenter (replace with your real one)
def syllable_segmenter(text):
    return text.split()

# Train BPE model
bpe = SyllableBPE(num_merges=50)
bpe.train(corpus, syllable_segmenter)

# Encode a new sentence
encoded = bpe.encode("ပ ညာ ရေး ဝန် ကြီး ဌာ န သည် အ ထက် တန်း ကျောင်း များ တွင် သင် ကြား မှု အ တွက် သင် ကြား ရေး အ ထောက် အ ကူ ပြု စာ အုပ် များ ကို ထုတ် ဝေ ခဲ့ သည်", syllable_segmenter)
print(encoded)  # Output: merged tokens based on BPE

['ပညာရေးဝန်ကြီးဌာနသည်အထက်တန်းကျောင်းများတွင်သင်ကြားမှုအတွက်သင်ကြားရေးအထောက်အကူပြုစာအုပ်များကိုထုတ်ဝေခဲ့သည်</w>']
